In [2]:
import fitz  # PyMuPDF
import pdfplumber
from PIL import Image
import io
import os

from gitdb.fun import chunk_size


def extract_all_from_pdf(pdf_path, output_dir="./extracted_data"):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    all_content = {"texts": [], "tables": [], "images": []}

    # 1. PyMuPDF(fitz)로 텍스트와 이미지 추출
    doc = fitz.open(pdf_path)
    # 2. pdfplumber로 테이블 추출 (레이아웃 보존 능력이 뛰어남)
    pdf_plumber = pdfplumber.open(pdf_path)

    for page_num in range(len(doc)):
        print(f"--- {page_num + 1} 페이지 처리 중 ---")

        # [텍스트 추출]
        page = doc.load_page(page_num)
        all_content["texts"].append(page.get_text())

        # [테이블 추출]
        plumber_page = pdf_plumber.pages[page_num]
        table = plumber_page.extract_table()
        if table:
            all_content["tables"].append(table)

        # [이미지 추출]
        for img_index, img in enumerate(page.get_images(full=True)):
            xref = img[0]
            base_image = doc.extract_image(xref)
            image_bytes = base_image["image"]
            image_ext = base_image["ext"]

            img_filename = f"page{page_num+1}_img{img_index+1}.{image_ext}"
            with open(os.path.join(output_dir, img_filename), "wb") as f:
                f.write(image_bytes)
            all_content["images"].append(img_filename)

    doc.close()
    pdf_plumber.close()
    return all_content

# 실행
content = extract_all_from_pdf("invest/invest.pdf")
print(f"\n추출 완료: 텍스트 {len(content['texts'])}개, 테이블 {len(content['tables'])}개, 이미지 {len(content['images'])}개")

--- 1 페이지 처리 중 ---
--- 2 페이지 처리 중 ---
--- 3 페이지 처리 중 ---

추출 완료: 텍스트 3개, 테이블 0개, 이미지 6개


In [10]:
import os

current_directory = os.getcwd()
fname = "invest.pdf"
fpath = os.path.join(os.path.dirname(current_directory), "ch14/invest/")

print("현재 스크립트의 위치:", current_directory)
print("pdf 위치:",fpath)

현재 스크립트의 위치: /Users/kks/Desktop/Laboratory/jocoding_langchain/ch14
pdf 위치: /Users/kks/Desktop/Laboratory/jocoding_langchain/ch14/invest/


In [14]:
def categorize_elements(raw_pdf_elements: list):
    """
    추출된 요소 리스트를 텍스트와 테이블로 분류합니다.

    :param raw_pdf_elements: 텍스트와 테이블(마크다운)이 섞인 리스트
    :return: (texts 리스트, tables 리스트) 튜플
    """
    texts = []
    tables = []
    for element in raw_pdf_elements:
        # Markdown 테이블 형식인지 간단히 확인 ( '|' 문자로 시작하고 끝나는지)
        # Markdown 테이블은 헤더와 구분선이 필수적으로 포함되므로 이를 기준으로 판단
        if isinstance(element, str) and element.strip().startswith("|") and "---" in element:
            tables.append(element)
        else:
            texts.append(element)
    return texts, tables

In [15]:
raw_pdf_elements = extract_all_from_pdf(fpath+fname)

texts, tables = categorize_elements(raw_pdf_elements)


--- 1 페이지 처리 중 ---
--- 2 페이지 처리 중 ---
--- 3 페이지 처리 중 ---


In [16]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=2000, chunk_overlap=200)

joined_texts  = " ".join(texts)
texts_2k_token = text_splitter.split_text(joined_texts)

print(len(texts_2k_token))
print(len(texts))

1
3
